# Generate 1024 parameter sets for Max Rintoul to effectivley sample his parameter space

### imports

In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sci
from scipy.stats import qmc

wrkdir = "/home/581/pjb581/for_MaxRintoul"

# print versions of packages
print("python version =",sys.version[:5])
print("numpy version =", np.__version__)
print("pandas version =", pd.__version__)
print("scipy version =", sci.__version__)
print("matplotlib version =", sys.modules[plt.__package__].__version__)

os.chdir(wrkdir)


python version = 3.10.
numpy version = 1.26.4
pandas version = 2.2.3
scipy version = 1.14.1
matplotlib version = 3.9.2


### find the Sobol parameter sequence for the parameter set

1. Temperature
2. Oxygen
3. Porosity
4. DIC
5. TA
6. Boundary layer thickness
7. POC flux
8. PIC flux

In [5]:
dim = 8
exps = 1024
sampler = qmc.Sobol(d=dim, scramble=True, seed=10)
sample_qmc = sampler.random(n=exps)  #here, n=256=2^8 
sample_qmc.shape


(1024, 8)

### Define the parameter ranges

In [6]:
# Define your data
data = {
    "T": [5.0, 34.0],
    "O2": [0.32, 450.0],
    "P": [40.0, 92.0],
    "DIC": [1500.0, 2500.0],
    "TA": [2200.0, 2500.0],
    "BL": [1e-4, 0.1],
    "fPOC": [0.11, 7.0],
    "fPIC": [2e-4, 0.62]
}

# Create DataFrame with variable names as rows
param_ranges = pd.DataFrame(data, index=["min", "max"]).T

# Display the DataFrame
print(param_ranges)


            min      max
T        5.0000    34.00
O2       0.3200   450.00
P       40.0000    92.00
DIC   1500.0000  2500.00
TA    2200.0000  2500.00
BL       0.0001     0.10
fPOC     0.1100     7.00
fPIC     0.0002     0.62


### Determine the parameter sets

In [12]:
param_sets = np.zeros((exps,dim))

# Loop over variables correctly
for ii, var in enumerate(param_ranges.index):
    param_sets[:, ii] = (
        param_ranges.loc[var, "min"]
        + (param_ranges.loc[var, "max"] - param_ranges.loc[var, "min"])
        * sample_qmc[:, ii]
    )

names = param_ranges.index
df = pd.DataFrame(param_sets, columns=names)
df

,T,O2,P,DIC,TA,BL,fPOC,fPIC
0,33.625886,361.139874,40.596738,1675.774775,2448.989363,0.002136,0.307213,0.189708
1,19.477119,126.119515,75.048578,2184.886507,2219.617299,0.072888,5.046243,0.489174
2,9.368693,251.034268,59.779416,1758.300880,2349.990370,0.089110,2.712931,0.404047
3,23.796895,7.885492,82.068925,2256.029730,2419.397507,0.035869,5.723048,0.081283
4,20.025189,327.489475,66.731915,2494.138729,2356.045527,0.021367,2.485381,0.339892
...,...,...,...,...,...,...,...,...
1019,20.064251,203.610556,50.305917,2279.227675,2410.998535,0.027035,2.764901,0.072901
1020,23.772648,361.654528,65.642232,2470.943280,2361.855560,0.016032,6.419166,0.434071
1021,9.393796,125.481340,82.306307,1981.027043,2292.395491,0.059176,2.540755,0.127351
1022,19.452958,250.464834,46.028037,2087.161059,2239.370142,0.078922,3.919875,0.222333


### Save the parameter sets to an excel spreadsheet

In [13]:
os.chdir(wrkdir)
df.to_excel('parameter_sets_1024.xlsx')
